<a href="https://colab.research.google.com/github/Luv4as/mdpi_ia4good/blob/master/GrafIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GrafIA

# 1. Instalação e Imports

In [33]:
# Instala a NOVA biblioteca do Google (v2) e Pydantic
!pip install -q google-genai pydantic transformers accelerate bitsandbytes neo4j sentencepiece gradio

import os
import json
import time
import torch
import gradio as gr
from google import genai
from pydantic import BaseModel, Field
from typing import List
from transformers import AutoModelForCausalLM, AutoTokenizer
from neo4j import GraphDatabase
from google.colab import userdata

print("✅ Dependências instaladas e bibliotecas importadas!")

✅ Dependências instaladas e bibliotecas importadas!


# 2. Configuração e Conexões

In [34]:
try:
    # Carrega as chaves
    NEO4J_URI = userdata.get('NEO4J_URI')
    NEO4J_USER = userdata.get('NEO4J_USERNAME').strip() # Added .strip()
    NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD').strip() # Added .strip()
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    driver.verify_connectivity()
    print("✅ Conexão Neo4j estabelecida com sucesso!")

except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    print("Verifique se você preencheu os 'Secrets' (chavinha na esquerda) corretamente.")

✅ Conexão Neo4j estabelecida com sucesso!


# 3. Carregar Modelo Local (LiquidAI)

In [35]:
print("⏳ Carregando modelo LiquidAI (LFM2-1.2B) na GPU...")

model_id = "LiquidAI/LFM2-1.2B-RAG" # Modelo otimizado

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype=torch.float16, # Importante para caber na T4
    trust_remote_code=True
)

print("✅ Modelo carregado na memória GPU!")

⏳ Carregando modelo LiquidAI (LFM2-1.2B) na GPU...
✅ Modelo carregado na memória GPU!


# 4. Funções do Pipeline (Lógica)

## 4.0 Schemas

In [36]:
class Node(BaseModel):
    id: str = Field(description="Nome único do conceito ou entidade")
    type: str = Field(description="Tipo do nó (ex: Concept, Tool, Method)", default="Concept")

class Edge(BaseModel):
    source: str = Field(description="Nó de origem")
    target: str = Field(description="Nó de destino")
    relation: str = Field(description="Relação em CAIXA_ALTA (ex: RELATES_TO)")

class KnowledgeGraph(BaseModel):
    nodes: List[Node]
    edges: List[Edge]

## 4.1 Extração

In [37]:
def step1_extrair_termos(texto):
    """Extrai APENAS as entidades principais (sujeitos)"""
    print(f"1. [LiquidAI] Analisando: {texto[:50]}...")

    if 'tokenizer' not in globals() or 'model' not in globals():
        return [texto]

    # Prompt reforçado para pegar KEYWORDS e não frases
    messages = [
        {"role": "system", "content": "Extract only the main technical KEYWORDS from the text. Ignore verbs and questions. Output JSON List."},
        {"role": "user", "content": 'Input: "O que é RAG?"'},
        {"role": "assistant", "content": '["RAG"]'},
        {"role": "user", "content": 'Input: "Explique como funciona a Fotossíntese"'},
        {"role": "assistant", "content": '["Fotossíntese"]'},
        {"role": "user", "content": f'Input: "{texto}"'}
    ]

    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

    outputs = model.generate(
        inputs, max_new_tokens=64, do_sample=False, temperature=0.1,
        pad_token_id=tokenizer.eos_token_id, attention_mask=torch.ones_like(inputs)
    )

    raw = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True).strip()

    # Função de limpeza recursiva (Mantida pois é boa)
    def extrair_strings(dados):
        lista = []
        if isinstance(dados, str): return [dados]
        if isinstance(dados, list):
            for item in dados: lista.extend(extrair_strings(item))
        if isinstance(dados, dict):
            for v in dados.values(): lista.extend(extrair_strings(v))
        return lista

    try:
        clean = raw.replace("```json", "").replace("```", "").split("Output:")[-1].strip()
        dados_brutos = json.loads(clean)
        termos_limpos = extrair_strings(dados_brutos)

        # Filtros extras: Remove palavras comuns que atrapalham a busca
        stopwords = ["o", "que", "é", "de", "para", "como", "explique", "conceito", "fundamental"]
        termos_finais = list(set([t for t in termos_limpos if len(t) > 2 and t.lower() not in stopwords]))

        print(f"   -> Termos Finais: {termos_finais}")
        return termos_finais if termos_finais else [texto]
    except:
        return [texto]

## 4.2 Expansão

In [38]:
def step2_gerar_grafo(termos, texto_original):
    """Usa Gemini 2.5 Lite (10 RPM) para economizar cota"""
    print(f"2. [Gemini 2.5 Lite] Criando estrutura estruturada...")

    # Modelo mais leve da sua lista
    MODELO_ESTRUTURA = "gemini-2.5-flash-lite"

    try:
        client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

        prompt = f"""
        Você é um arquiteto de Knowledge Graphs.
        Contexto do Usuário: "{texto_original}"
        Entidades Chave: {termos}

        Crie um grafo conectando esses conceitos. Use relações lógicas.
        """

        # Retry Automático
        for tentativa in range(3):
            try:
                response = client.models.generate_content(
                    model=MODELO_ESTRUTURA,
                    contents=prompt,
                    config={
                        "response_mime_type": "application/json",
                        "response_schema": KnowledgeGraph,
                    },
                )
                return response.parsed.model_dump()

            except Exception as e:
                if "429" in str(e) or "RESOURCE_EXHAUSTED" in str(e):
                    print(f"⚠️ Cota do {MODELO_ESTRUTURA} cheia. Esperando 30s...")
                    time.sleep(30)
                else:
                    print(f"❌ Erro no Step 2: {e}")
                    return None
        return None

    except Exception as e:
        print(f"❌ Erro geral Step 2: {e}")
        return None

## 4.3 Neo4j

In [39]:
def step3_salvar_neo4j(dados_dict):
    """Salva no Neo4j"""
    if 'driver' not in globals() or not driver or not dados_dict: return
    print(f"3. [Neo4j] Salvando dados...")
    try:
        with driver.session() as session:
            for node in dados_dict.get('nodes', []):
                session.run("MERGE (n:Concept {name: $id}) SET n.type = $type", id=node['id'], type=node.get('type', 'Concept'))
            for edge in dados_dict.get('edges', []):
                rel = edge['relation'].upper().replace(" ", "_").replace("-", "_")
                # type: ignore
                session.run(f"MATCH (s {{name: $s}}), (t {{name: $t}}) MERGE (s)-[:{rel}]->(t)", s=edge['source'], t=edge['target'])
    except Exception as e:
        print(f"❌ Erro Step 3: {e}")

In [40]:
def step4_consultar_neo4j(termos):
    """Lê do Neo4j com Validação de Tipo"""
    print(f"4. [Neo4j] Buscando contexto...")
    if 'driver' not in globals() or not driver: return ""
    termos_seguros = [str(t) for t in termos if isinstance(t, (str, int))]
    if not termos_seguros: return ""

    try:
        query = """
        UNWIND $termos as t
        MATCH (n:Concept)-[r]-(m)
        WHERE toLower(n.name) CONTAINS toLower(t)
        RETURN DISTINCT n.name, type(r), m.name LIMIT 20
        """
        ctx = ""
        with driver.session() as session:
            res = session.run(query, termos=termos_seguros)
            for rec in res:
                ctx += f"({rec['n.name']}) --[{rec['type(r)']}]--> ({rec['m.name']})\n"
        return ctx
    except Exception as e:
        print(f"❌ Erro Step 4: {e}")
        return ""

## 4.4 Resposta

In [41]:
def step5_resposta_final(texto, contexto):
    """
    Gera a resposta com Persona de Mentor Socrático.
    Corrigido para evitar loops quando o usuário pede a explicação.
    """

    # Modelo escolhido
    MODELO_RESPOSTA = "gemini-2.5-flash"

    print(f"5. [{MODELO_RESPOSTA}] O Mentor está analisando a trilha de aprendizado...")

    # Debug no terminal
    print(f"\n{'='*40}\n🔍 CONTEXTO DO GRAFO (Memória):\n{contexto}\n{'='*40}\n")

    if not contexto.strip():
        contexto = "O grafo está vazio. Diga ao aluno que você está mapeando este assunto pela primeira vez."

    try:
        client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

        # --- PROMPT PEDAGÓGICO AVANÇADO V2 (CORRIGIDO) ---
        prompt_mentor = f"""
        JAILBREAK INSTRUCTION: Você é um MENTOR SOCRÁTICO.

        INPUT DO ALUNO: "{texto}"

        BASE DE CONHECIMENTO (Grafo):
        {contexto}

        SUA MISSÃO (Siga a lógica abaixo):

        1. **ANALISE A INTENÇÃO:** - O aluno está fazendo uma nova pergunta do zero?
           - OU o aluno está respondendo à sua sugestão anterior (ex: "Explique isso", "Não sei", "Comece por aí")?

        2. **CENÁRIO A (Continuação/Aceite):** - Se o aluno pediu para explicar o conceito base/fundamental: **EXPLIQUE-O IMEDIATAMENTE**.
           - Use o grafo para definir esse conceito base.
           - Após explicar o base, mostre como ele se conecta ao conceito original que ele queria saber.

        3. **CENÁRIO B (Pergunta Nova/Complexa):**
           - Se for um tópico novo e complexo (com "pais" ou "pré-requisitos" no grafo): **NÃO EXPLIQUE AINDA**.
           - Identifique o pré-requisito (nó pai).
           - Diga: "Para entender [ALVO], precisamos dominar [PRÉ-REQUISITO]."
           - Pergunte se ele já conhece [PRÉ-REQUISITO].

        4. **CENÁRIO C (Simples):**
           - Se não houver dependências complexas, explique didaticamente.

        TOM DE VOZ:
        - Didático, paciente e estruturado.
        - Não repita a pergunta de sondagem se o aluno já pediu a explicação.
        """

        # Retry Lógico
        for tentativa in range(3):
            try:
                response = client.models.generate_content(
                    model=MODELO_RESPOSTA,
                    contents=prompt_mentor
                )
                return response.text

            except Exception as e:
                erro_str = str(e)
                if "429" in erro_str or "RESOURCE_EXHAUSTED" in erro_str:
                    print(f"⚠️ Cota cheia no Step 5 (Tentativa {tentativa+1}). Esperando 30s...")
                    time.sleep(30)
                else:
                    return f"Erro na IA: {e}"

        return "O Mentor está indisponível (Cota). Tente em 1 min."

    except Exception as e:
        return f"Erro técnico no Mentor: {e}"

# 5. Interface Gradio (UI)

In [42]:
# --- 5. INTERFACE GRADIO (MEMÓRIA CORRIGIDA) ---

def pipeline_completo(message, history):
    # --- 1. MEMÓRIA DE CONTEXTO ---
    texto_para_analise = message

    if history:
        # Pega a última interação (User, AI)
        ultima_interacao = history[-1]
        ultima_pergunta_user = ultima_interacao[0]
        ultima_resposta_ia = ultima_interacao[1]

        # ESTRATÉGIA DE CONTINUIDADE:
        # Em vez de pegar os últimos caracteres, pegamos um resumo focado.
        # Se a resposta anterior for muito longa, pegamos o início (onde geralmente está a definição) e o fim (pergunta socrática).
        if len(ultima_resposta_ia) > 300:
            resumo_ia = ultima_resposta_ia[:150] + " ... " + ultima_resposta_ia[-150:]
        else:
            resumo_ia = ultima_resposta_ia

        # Monta um prompt claro para o LiquidAI entender a referência "disso/daquilo"
        texto_para_analise = f"""
        Histórico Recente:
        Usuário: {ultima_pergunta_user}
        Mentor: {resumo_ia}

        Nova Pergunta do Usuário (resolva referências como 'isso' ou 'ele'): {message}
        """

    # --- 2. EXTRAÇÃO ---
    termos = step1_extrair_termos(texto_para_analise)
    yield f"🧠 [LiquidAI] Analisando: `{message}`\n(Contexto detectado: {termos})..."

    # --- 3. VERIFICAÇÃO INICIAL ---
    contexto = step4_consultar_neo4j(termos)

    # --- 4. APRENDIZADO (Se necessário) ---
    if not contexto:
        # Verifica se realmente precisa aprender ou se foi erro de extração
        if len(termos) > 0 and len(termos[0]) > 2:
            yield f"🧠 [LiquidAI] Tópico `{termos}` novo.\n🕸️ Consultando Gemini para criar o Grafo..."

            dados_grafo = step2_gerar_grafo(termos, message) # Usa a mensagem original aqui para o Gemini entender a dúvida real

            if dados_grafo:
                step3_salvar_neo4j(dados_grafo)
                contexto = step4_consultar_neo4j(termos)
                yield f"🧠 [LiquidAI] Grafo criado! Gerando aula..."
            else:
                contexto = "Erro: Falha na API ao criar grafo."
        else:
             contexto = "Aviso: Termos muito vagos para busca no grafo."

    # --- 5. RESPOSTA FINAL ---
    resposta = step5_resposta_final(message, contexto)
    yield resposta

# Configura a interface com visual LIMPO
demo = gr.ChatInterface(
    fn=pipeline_completo,
    title="🤖 GrafIA",
    description="Mentor Socrático. O sistema aprende (cria grafos) e ensina passo a passo.",
    examples=["O que é RAG?", "Explique Docker containers", "Como funciona a fotossíntese?"],
    theme="soft"
)

print("✅ Interface Pronta! Pode rodar o Launch.")

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


✅ Interface Pronta! Pode rodar o Launch.


# 6. Execução (Launch)

In [43]:
print("🚀 Iniciando servidor Gradio...")

# share=True gera o link público temporário (ex: https://...gradio.live)
# debug=True mostra os erros no console do Colab se algo quebrar
demo.queue().launch(share=True, debug=True)

🚀 Iniciando servidor Gradio...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9bc332db748b015d3c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1. [LiquidAI] Analisando: Explique Docker containers...
   -> Termos Finais: ['Docker containers']
4. [Neo4j] Buscando contexto...
2. [Gemini 2.5 Lite] Criando estrutura estruturada...
3. [Neo4j] Salvando dados...
4. [Neo4j] Buscando contexto...
5. [gemini-2.5-flash] O Mentor está analisando a trilha de aprendizado...

🔍 CONTEXTO DO GRAFO (Memória):
(Docker containers) --[ENABLES]--> (Resource Management)
(Docker containers) --[ENABLES]--> (Portability)
(Docker containers) --[IS_A_PART_OF]--> (Containerization)
(Docker containers) --[PROVIDES]--> (Isolation)
(Docker containers) --[MANAGED_BY]--> (Docker Engine)
(Docker containers) --[IMPLEMENTS]--> (OS-level virtualization)


⚠️ Cota cheia no Step 5 (Tentativa 1). Esperando 30s...
⚠️ Cota cheia no Step 5 (Tentativa 2). Esperando 30s...
⚠️ Cota cheia no Step 5 (Tentativa 3). Esperando 30s...
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9bc332db748b015d3c.gradio.live
